<img src='https://hammondm.github.io/hltlogo1.png' style="float:right">

Linguistics 578<br>
Fall 2023<br>
Hammond

## Things to remember about any homework assignment:

1. For this assignment, you will edit this jupyter notebook and turn it in. Do not turn in pdf files or separate `.py` files.
1. Late work is not accepted.
1. Given the way I grade, you should try to answer *every* question, even if you don't like your answer or have to guess.
1. You may *not* use `python` modules that we have not already used in class.
1. You may certainly talk to your classmates about the assignment, but everybody must turn in *their own* work. It is not acceptable to turn in work that is essentially the same as the work of classmates.
1. All code must run. It doesn't have to be perfect, it may not do all that you want it to do, but it must run without error.
1. Code must run in reasonable time. Assume that if it takes more than *5 minutes* to run (on your machine), that's too long.
1. Please do not add, remove, or copy autograded cells.
1. Make sure to select `restart, run all cells` from the `kernel` menu when you're done and before you turn this in!

***

***my name***: Ashwin Raj

***people I talked to about the assignment***: [put your answer here]

***

## Homework #4

Here are the imports. Please do not import anything else.

In [1]:
import graphviz,re
import pywrapfst as fst
import numpy as np

1. Using `pywrapfst`, create a symbol table for the (lowercase) English alphabet plus $\epsilon$ and `#`. (The second symbol added should be `#`.)

In [2]:
st = fst.SymbolTable()
eng = "abcdefghijklmnopqrstuvwxyz"
letters = []
letters.append("<epsilon>")
letters.append("#")
for char in eng:
    letters.append(char)
for letter in letters:
    st.add_symbol(letter)

In [3]:
assert st.num_symbols() == 28

In [4]:
assert st.find(0) == '<epsilon>'

In [5]:
assert st.find(1) == '#'

2. We'll now build a character *bigram model* based on the *Alice in Wonderland* text. The following function should strip the Gutenberg information at the beginning and end of the file, convert everything to lowercase, convert everything except lowercase letters to space, and return a list of lowercase words.

In [6]:
def getwords(alice):
    '''tokenizes the Alice text
    args:
        alice: the location of the Gutenberg alice file
    returns
        a list of lowercase words
    '''
    # YOUR CODE HERE
    f = open(alice,'r')
    t = f.read()
    f.close()
    t = t[1435:]
    t = t[:-18590]
    t = t.lower()

    t = re.sub(r"[^a-z]+"," ",t)

    words = t.split()

    return words

In [7]:
words = getwords('alice.txt')
assert words[:12] == [
    'chapter', 'i', 'down', 'the', 'rabbit', 'hole',
    'alice', 'was', 'beginning', 'to', 'get', 'very'
]

In [8]:
assert words[-5:] == ['happy', 'summer', 'days', 'the', 'end']

In [9]:
assert len(words) == 27336

3. We now write a function that will return a list of counts for letter unigrams and letter bigrams. You'll need to pad each word with `#` on each side before doing this.

In [10]:
def getcounts(wds):
    '''get letter unigram and bigram counts from a list of words
    args:
        words: a list of words
    returns
        unigrams: a dictionary from letters to counts
        bigrams: a dictionary from letter pairs to counts
    '''
    # YOUR CODE HERE
    unigrams = {}
    bigrams = {}
    new_words = ["#" + word + "#" for word in wds]
    for word in new_words:
        for char in word:
            if char not in unigrams:
                unigrams[char] = 1
            else:
                unigrams[char]+=1
    for word in new_words:
        for i in range(len(word)-1):
            bigram = word[i] + word[i+1]
            if bigram not in bigrams:
                bigrams[bigram] = 1
            else:
                bigrams[bigram]+=1
                
    return unigrams,bigrams
    


In [11]:
ugs,bgs = getcounts(words)
assert len(ugs) == 27

In [12]:
assert len(bgs) == 427

In [13]:
assert bgs['ab'] == 214

In [14]:
assert ugs['#'] == 54672

In [15]:
assert ugs['q'] == 209

4. Let's now write a function that takes our unigram and bigram counts and creates a dictionary of log conditional probabilities.

In [16]:
def makecondprobs(unigrams,bigrams):
    '''calculate conditional probabilities
    args:
        unigrams: dictionary of unigram counts
        bigrams: dictionary of bigram counts
    returns:
        a dictionary of log conditional probabilities
    '''
    # YOUR CODE HERE
    conditional_probs = {}
    for bigram, bigram_count in bigrams.items():
        unigram1, unigram2 = bigram[0], bigram[1] 
        if unigram1 in unigrams:
            unigram1_count = unigrams[unigram1]
        else:
            unigram1_count = 0
        numerator = bigram_count
        denominator = unigram1_count
        prob = (numerator) / (denominator)
        log_prob = np.log(prob)
        conditional_probs[bigram] = log_prob

    return conditional_probs

In [17]:
cp = makecondprobs(ugs,bgs)
assert len(cp) == 427

In [18]:
assert np.isclose(cp['ab'],-3.71,atol=.01)

In [19]:
total = 0
for bigram in cp:
    if bigram[0] == 'a':
        total += np.exp(cp[bigram])
assert np.isclose(total,1.0,atol=.0001)

5. We now create a WFSA for this model using the symbol table you created in question #1 and the dictionary we just created. The state geometry is critical here. There will be a start state, which you can think of as the `#` on the left. From there, the first arc takes you to various letter states. The probabilities of those arcs will be the conditional probabilities of the different letters based on the initial `#`. Now from each letter state you can go to any other letter state or the final `#`. In each case, the relevant probability is the conditional probability of the second letter/`#` based on the first letter. The second `#` state is the only final state. Note that there will be no arc corresponding to the first `#`.

In [20]:
def makeWFSA(tab,dic):
    '''create a WFSA that encodes a bigram model
    args:
        tab: a symbol table including all letters
            and boundary markers
        dic: a dictionary of the conditional probabilities
    returns:
        a WFSA
    '''
    # YOUR CODE HERE
    
    c1 = fst.Compiler(isymbols=tab, osymbols=tab, arc_type="log",keep_isymbols=True,keep_osymbols=True)
    boundary_id = tab.find("#")
    num_symbols = tab.num_symbols()
    state_map = {}


    state_index = 1  
    for i in range(2, num_symbols):  
        letter_id = i
        letter = tab.find(letter_id)
        prob = dic.get("#"+letter,0.0)
        if prob != 0.0:
            #state_write = "0 " + state_index + " " + 
            if letter not in state_map:
                state_map[letter] = state_index
            c1.write(f"0 {state_map[letter]} {letter} {letter} {prob}\n")
            state_index += 1

    for i in range(2, num_symbols):  
        letter_id=i
        letter = tab.find(letter_id)
        for j in range(2, num_symbols):  
            next_letter_id = j
            next_letter = tab.find(next_letter_id)
            next_prob = dic.get(letter+next_letter,0.0)
            if next_prob != 0.0:
                
                next_state = j
                cur_state = i
                if letter in state_map:
                    cur_state = state_map[letter]
                if next_letter in state_map:
                    next_state = state_map[next_letter]
                if letter != "#" and next_letter!="#":
                    c1.write(f"{cur_state} {next_state} {next_letter} {next_letter} {next_prob}\n")
                state_index += 1

        final_prob = dic.get(letter+"#",0.0)
        if final_prob != 0.0:
            cur_state = i
            if letter in state_map:
                cur_state = state_map[letter]
            c1.write(f"{cur_state} {1000} # # {final_prob}\n")
    c1.write(str(1000))

    w = c1.compile()
    return w

Number of states in wfsa: 28
0
0 2 2 -2.77757621 1
0 3 3 -3.98252988 2
0 4 4 -4.09346676 3
0 5 5 -4.14869213 4
0 6 6 -4.93275595 5
0 7 7 -4.28637075 6
0 8 8 -4.54953289 7
0 9 9 -3.53698897 8
0 10 10 -3.31371975 9
0 11 11 -6.10508585 10
0 12 12 -5.33695316 11
0 13 13 -4.21578312 12
0 14 14 -4.04113245 13
0 15 15 -4.53749514 14
0 16 16 -3.62223053 15
0 17 17 -4.6727376 16
0 18 18 -5.6833601 17
0 19 19 -4.62871122 18
0 20 20 -2.9926641 19
0 21 21 -2.45762634 20
0 22 22 -5.32185841 21
0 23 23 -5.26365995 22
0 24 24 -3.37274313 23
0 25 25 -9.81049442 24
0 26 26 -4.59194231 25
0 27 27 -10.2159595 26
1
1 3 3 -3.71516633 2
1 4 4 -4.02489662 3
1 5 5 -2.99209762 4
1 7 7 -4.93800783 6
1 8 8 -4.00596857 7
1 9 9 -5.86226654 8
1 10 10 -2.50885987 9
1 11 11 -6.59623575 10
1 12 12 -4.2528286 11
1 13 13 -2.24811077 12
1 14 14 -3.87165618 13
1 15 15 -1.70026338 14
1 16 16 -7.98253012 15
1 17 17 -4.38979435 16
1 19 19 -2.52436399 18
1 20 20 -2.2798593 19
1 21 21 -2.02152491 20
1 22 22 -4.75040913 21
1 23

In [21]:
wfsa = makeWFSA(st,cp)
assert type(wfsa) == fst.MutableFst

In [22]:
assert wfsa.num_states() == 28

In [23]:
c2 = fst.Compiler(
    isymbols=st,
    osymbols=st,
    keep_isymbols=True,
    keep_osymbols=True,
    arc_type='log'
)
c2.write('0 1 h h')
c2.write('1 2 a a')
c2.write('2 3 t t')
c2.write('3 4 # #')
c2.write('4')
f2 = c2.compile()
f3 = fst.compose(f2,wfsa)
assert f3.num_states() == 5

In [24]:
f3s = str(f3).split('\n')[:-2]
val = np.exp(sum([float(s.split('\t')[-1]) for s in f3s]))
assert np.isclose(val,0.00019,atol=.001)